<a href="https://colab.research.google.com/github/tttequila/Kaggle_20Q/blob/main/LLM_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configuring your kaggle token, see more details in the [**Configure your API key**](https://ai.google.dev/gemma/docs/setup) section

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%%bash
mkdir ~/.kaggle
# change the first path to your path of kaggle.json
cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

In [3]:
%%bash
pip install -q -U torch immutabledict sentencepiece

# pip install -q -U kerasnlp
# pip install -q -U keras>=3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.1 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.


### Loading mode

In [4]:
# %%bash
!git clone https://github.com/google/gemma_pytorch.git

# !mkdir /kaggle/working/gemma/
# !mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 222 (delta 90), reused 87 (delta 55), pack-reused 68
Receiving objects: 100% (222/222), 2.19 MiB | 15.98 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [5]:
# login the kaggle (need to store you kaggle.json to your google dirve)
import kagglehub
kagglehub.login()

In [6]:
# Choose variant and machine type
VARIANT = '7b-it-quant'
MACHINE_TYPE = 'cuda'

import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'
# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

Attaching model 'google/gemma/pyTorch/7b-it-quant' to your Colab notebook...


In [ ]:
# import sys
# sys.path.append("/content/gemma_pytorch/gemma")
# print(sys.path)
# from gemma.config import get_config_for_7b, get_config_for_2b
# from gemma.model import GemmaForCausalLM, GemmaModel
# import contextlib, torch


In [7]:
import sys
sys.path.append("gemma_pytorch/gemma")
sys.path.append("gemma_pytorch")
print(sys.path)
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM, GemmaModel
import contextlib, torch


['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', 'gemma_pytorch/gemma', 'gemma_pytorch']


In [8]:

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

## Loading model

In [9]:
import torch
import gemma

# Set up model config.
model_config = get_config_for_7b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT
# model_config.architecture = gemma.config.Architecture.GEMMA_2

with _set_default_tensor_type(model_config.get_dtype()):
  device = torch.device(MACHINE_TYPE)
  model = GemmaForCausalLM(model_config)
  model.load_weights(ckpt_path)
  model = model.to(device).eval()

## Play with Gemma

### format your prompts

In [ ]:
# Chat templates
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt='What is a good place for travel in the US?'
    )
)
print('Chat prompt:\n', prompt)

### Chat & Response

In [ ]:
import time
# generate with formatted prompts
start = time.time()
response_1 = model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=100,
)
print(f"[response in {(time.time()-start):.2f}s]\n{response_1}")

[response in 40.20s]
The best place to travel in the US depends on your interests and preferences. Here are a few popular destinations:

**For city lovers:**

* **New York, NY:** A vibrant and historical city with a diverse culture.
* **Los Angeles, CA:** A beautiful and glamorous city with a variety of neighborhoods to explore.
* **Chicago, IL:** A friendly city with a rich history and iconic architecture.
* **San Francisco, CA:** A historic city with a stunning Golden


In [10]:
# or directly talk to Gemma
start = time.time()
response_2 = model.generate(
    'Write a poem about an llm writing a poem.',
    device=device,
    output_len=2000,
)
print(f"[response in {(time.time()-start):.2f}s]\n{response_2}")

NameError: name 'time' is not defined

### applying formatter

### documents

<details>
  <summary> model </summary>
  
  - `self.forward()`: getting next token and corresponding logits
  - `self.generate()`: see summary below. May need to be rewriten if we wanna get the cumulative logits for the whole sentence  


</details>



<details>
  <summary> model.generate() </summary>
  
  - **prompts** | `Union[str, Sequence[str]]`: Your prompts
  - **device** | `Any`: Devices
  - **output_len** | `int`: max output length
  - **temperature** | `Union[float, None]`: temperature degree, controlling how variant its response could be  
  - **top_p** | `float`:
  - **top_k** | `int`:

  regarding temperature, top_p and top_k, check this [link](https://blog.csdn.net/REfusing/article/details/137866583)

</details>

In [ ]:
import itertools
from typing import Iterable

class formatter:
    
    '''
        formatter class to format the prompt text for the model. 
        A general idea is 
    '''
    
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'
        
    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._template_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._template_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self._all_pompt = ''
        # self.reset()

    def __call__(self, prompt):
        return self._all_prompt
        
    def reset(self):
        if self._system_prompt:
            self._all_prompt += self._template_model.format(self._system_prompt)
        
    def add_user_round(self, user_prompt: str | None):
        self._all_prompt

    def add_agent_round(self):
        raise NotImplementedError
    
    def add_few_shot_examples(self):
        raise NotImplementedError
    
    def add_system_prompt(self):
        raise NotImplementedError
    
    def add_existing_rounds(self):
        raise NotImplementedError
    
    def add_new_round(self):
        raise NotImplementedError
    
    def formate_MCQA(self):
        raise NotImplementedError
    

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_se